In [47]:
import pandas as pd
df = pd.read_csv("compas-scores-two-years.csv")
df = df.loc[df["race"].isin(["African-American", "Caucasian"])]
df["race"].replace(['African-American', 'Caucasian'],[0, 1], inplace=True)
df["sex"] = pd.get_dummies(df["sex"])["Female"]
le={"Low":0,"Medium":1,"High":2}
df["score_text"]= df["score_text"].replace(le)
df.columns.values

array(['id', 'name', 'first', 'last', 'compas_screening_date', 'sex',
       'dob', 'age', 'age_cat', 'race', 'juv_fel_count', 'decile_score',
       'juv_misd_count', 'juv_other_count', 'priors_count',
       'days_b_screening_arrest', 'c_jail_in', 'c_jail_out',
       'c_case_number', 'c_offense_date', 'c_arrest_date',
       'c_days_from_compas', 'c_charge_degree', 'c_charge_desc',
       'is_recid', 'r_case_number', 'r_charge_degree',
       'r_days_from_arrest', 'r_offense_date', 'r_charge_desc',
       'r_jail_in', 'r_jail_out', 'violent_recid', 'is_violent_recid',
       'vr_case_number', 'vr_charge_degree', 'vr_offense_date',
       'vr_charge_desc', 'type_of_assessment', 'decile_score.1',
       'score_text', 'screening_date', 'v_type_of_assessment',
       'v_decile_score', 'v_score_text', 'v_screening_date', 'in_custody',
       'out_custody', 'priors_count.1', 'start', 'end', 'event',
       'two_year_recid'], dtype=object)

In [5]:
len(df)

6150

In [56]:
s = df["race"]
y = df["two_year_recid"]
X = pd.DataFrame(df, columns=["race","age","priors_count","juv_fel_count","juv_misd_count","juv_other_count",'decile_score','score_text',"sex"])

In [57]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=879, random_state=42, stratify=y)
(X_train.shape, y_train.shape), (X_test.shape, y_test.shape)

(((5271, 9), (5271,)), ((879, 9), (879,)))

In [58]:
X_train, X_valid, y_train, y_valid=train_test_split(X_train,y_train,test_size=879, random_state=42, stratify=y_train)
(X_train.shape, y_train.shape), (X_valid.shape, y_valid.shape)

(((4392, 9), (4392,)), ((879, 9), (879,)))

#Baseline

In [59]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X_train, y_train)

In [60]:
clf.score(X_test,y_test)

0.7007963594994312

In [70]:
from sklearn.metrics import classification_report
y_pred=clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.70      0.78      0.74       469
           1       0.71      0.61      0.65       410

    accuracy                           0.70       879
   macro avg       0.70      0.69      0.70       879
weighted avg       0.70      0.70      0.70       879



In [84]:
#calibration score
from sklearn import metrics
abs(metrics.recall_score(y_test, y_pred, pos_label=0)-metrics.recall_score(y_test, y_pred, pos_label=1))

0.17519891830048362